# BigGAN nonlinear walk - Shift X

## 1. Setup

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

In [ ]:
%cd ..

Pick output directory.

In [ ]:
output_dir = 'notebooks/models/biggan_nonlinear_shiftx'

Pick learning rate and number of samples.

In [ ]:
lr = 0.0001
num_samples = 20000

Pick step size and number of steps

In [ ]:
eps = 25
num_steps = 5

## 2. Create Graph and initialize

In [ ]:
# make output directory
import os
os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'output'), exist_ok=True)

In [ ]:
module_path = 'https://tfhub.dev/deepmind/biggan-256/2'

import io
import IPython.display
import numpy as np
import PIL.Image
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import time
from resources import tf_lpips_pkg as lpips_tf


tf.reset_default_graph()
print('Loading BigGAN module from:', module_path)
module = hub.Module(module_path)

inputs = {k: tf.placeholder(v.dtype, v.get_shape().as_list(), k)
          for k, v in module.get_input_info_dict().items()}
output = module(inputs)

print('Inputs:\n', '\n'.join(
    '  {}: {}'.format(*kv) for kv in inputs.items()))
print('Output:', output)

input_z = inputs['z']
input_y = inputs['y']
input_trunc = inputs['truncation']
dim_z = input_z.shape.as_list()[1]
vocab_size = input_y.shape.as_list()[1]

In [ ]:
# input placeholders
Nsliders = 1
z = tf.placeholder(tf.float32, shape=(None, dim_z))
y = tf.placeholder(tf.float32, shape=(None, vocab_size))
truncation = tf.placeholder(tf.float32, shape=None)

# original output
inputs_orig = {u'y': y,
               u'z': z,
               u'truncation': truncation}
outputs_orig = module(inputs_orig)

img_size = outputs_orig.shape[1].value
num_channels = outputs_orig.shape[-1].value

# output placeholders
target = tf.placeholder(tf.float32, shape=(
    None, img_size, img_size, num_channels))
mask = tf.placeholder(tf.float32, shape=(
    None, img_size, img_size, num_channels))

# set walk parameters
alpha = tf.placeholder(tf.int32, shape=())
T1 = tf.keras.layers.Dense(dim_z, input_shape=(None, dim_z), kernel_initializer='RandomNormal', bias_initializer='zeros', activation=tf.nn.relu)
T2 = tf.keras.layers.Dense(dim_z, input_shape=(None, dim_z), kernel_initializer='RandomNormal', bias_initializer='zeros')
T3 = tf.keras.layers.Dense(dim_z, input_shape=(None, dim_z), kernel_initializer='RandomNormal', bias_initializer='zeros', activation=tf.nn.relu)
T4 = tf.keras.layers.Dense(dim_z, input_shape=(None, dim_z), kernel_initializer='RandomNormal', bias_initializer='zeros')
# forward transformation
out_f = []
z_prev = z
z_norm = tf.norm(z, axis=1, keepdims=True)
for i in range(1, num_steps + 1):
    z_step = z_prev + T2(T1(z_prev))
    z_step_norm = tf.norm(z_step, axis=1, keepdims=True)
    z_step = z_step * z_norm / z_step_norm
    out_f.append(z_step)
    z_prev = z_step

# reverse transformation
out_g = []
z_prev = z
z_norm = tf.norm(z, axis=1, keepdims=True)
for i in range(1, num_steps + 1):
    z_step = z_prev + T4(T3(z_prev))
    z_step_norm = tf.norm(z_step, axis=1, keepdims=True)
    z_step = z_step * z_norm / z_step_norm
    out_g.append(z_step)
    z_prev = z_step
out_g.reverse() # flip the reverse transformation

# w has shape (2*num_steps + 1, batch_size, dim_z)
# elements 0 to num_steps are the reverse transformation, in reverse order
# elements num_steps + 1 to 2*num_steps + 1 are the forward transformation
# element num_steps is no transformation
w = tf.stack(out_g+[z]+out_f)

# w is already z+f(z) so we can just index into w
z_new = w[alpha,  :, :]

transformed_inputs = {u'y': y,
                      u'z': z_new,
                      u'truncation': truncation}
transformed_output = module(transformed_inputs)

# losses
loss = tf.losses.compute_weighted_loss(tf.square(
    transformed_output-target), weights=mask)
loss_lpips = tf.reduce_mean(lpips_tf.lpips(
    mask*transformed_output, mask*target, model='net-lin', net='alex'))

# train op 
# change to loss_lpips to loss to optimize l2 loss
train_step = tf.train.AdamOptimizer(lr).minimize(
    loss_lpips, var_list=tf.trainable_variables(scope=None), name='AdamOpter')

In [ ]:
initializer = tf.global_variables_initializer()
config = tf.ConfigProto(log_device_placement=False)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(initializer)
saver = tf.train.Saver(tf.trainable_variables(scope=None))

## 3. Define Target Operation

In [ ]:
def get_target_np(outputs_zs, alpha, show_img=False, show_mask=False):
    
    mask_out = np.ones(outputs_zs.shape)
 
    if alpha == 0:
        return outputs_zs, mask_out
    
    M = np.float32([[1,0,alpha],[0,1,0]])
    target_fn = np.zeros(outputs_zs.shape)
    
    for i in range(outputs_zs.shape[0]):
        target_fn[i,:,:,:] = cv2.warpAffine(outputs_zs[i,:,:,:], M, (img_size, img_size))
        mask_out[i,:,:,:] = cv2.warpAffine(mask_out[i,:,:,:], M, (img_size, img_size))

    mask_out[np.nonzero(mask_out)] = 1.
    assert(np.setdiff1d(mask_out, [0., 1.]).size == 0)

    return target_fn, mask_out

## 4. Train walk

In [ ]:
# define sampling operations
from graphs.biggan.graph_util import *

In [ ]:
# This can be train.py

import logging
import sys
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(output_dir, 'train')),
        logging.StreamHandler(sys.stdout)
    ])
logger = logging.getLogger()

loss_vals = []


# train
def train(saver):
    trunc=1.0
    noise_seed=0
    zs = truncated_z_sample(num_samples, trunc, noise_seed)
    ys = np.random.randint(0,vocab_size,size=zs.shape[0])
    ys = one_hot_if_needed(ys, vocab_size)

    Loss_sum = 0
    n_epoch = 1
    Loss_sum_iter = 0
    optim_iter = 0
    batch_size = 4
    for epoch in range(n_epoch):
        for batch_start in range(0, num_samples, batch_size):
            start_time = time.time()

            alpha_val = np.random.randint(-num_steps, num_steps+1)
            alpha_to_graph = alpha_val + num_steps
            alpha_to_target = alpha_val * eps

            s = slice(batch_start, min(num_samples, batch_start + batch_size))

            feed_dict_out = {z: zs[s], y: ys[s], truncation: trunc}
            out_zs = sess.run(outputs_orig, feed_dict_out)

            target_fn, mask_out = get_target_np(out_zs, alpha_to_target)
            
            feed_dict = {z: zs[s], y: ys[s], truncation: trunc, alpha: alpha_to_graph, target: target_fn, mask: mask_out}
            
            curr_loss, _ = sess.run([loss, train_step], feed_dict=feed_dict)
            Loss_sum = Loss_sum + curr_loss
            Loss_sum_iter = Loss_sum_iter + curr_loss
            
            elapsed_time = time.time() - start_time

            logger.info('T, epc, bst, lss, a: {}, {}, {}, {}, {}'.format(elapsed_time, epoch, batch_start, curr_loss, alpha_to_target))


            if (optim_iter % 100 == 0) and (optim_iter > 0):
                saver.save(sess, './{}/{}/model_{}.ckpt'.format(output_dir, 'output', optim_iter*batch_size), write_meta_graph=False, write_state=False)

            if (optim_iter % 100 == 0) and (optim_iter > 0):
                loss_vals.append(Loss_sum_iter/(100*batch_size))
                Loss_sum_iter = 0
                print('Loss:', loss_vals)
                
            optim_iter = optim_iter+1
            
    if optim_iter > 0:
        print('average loss with this metric: ', Loss_sum/(optim_iter*batch_size))
    saver.save(sess, "./{}/{}/model_{}.ckpt".format(output_dir, 'output', optim_iter*batch_size), write_meta_graph=False, write_state=False)

In [ ]:
train(saver)

## 5. Visualizations

In [ ]:
from utils.image import imgrid, imshow

In [ ]:
# To restore previous w:
saver.restore(sess, "./{}/{}/model_{}.ckpt".format(output_dir, 'output', 20000))

In [ ]:
# test: show imgs 
# this can be test.py

category = 207

a = np.linspace(-200, 200, 9)

trunc=0.5
noise_seed=20   
num_samples_vis = 6
batch_size = 1

zs = truncated_z_sample(num_samples_vis, trunc, noise_seed)
ys = np.array([category] * zs.shape[0])
ys = one_hot_if_needed(ys, vocab_size)

for batch_num, batch_start in enumerate(range(0, num_samples_vis, batch_size)):

    ims = []
    targets = []

    s = slice(batch_start, min(num_samples, batch_start + batch_size))

    input_test = {y: ys[s],
                  z: zs[s],
                  truncation: trunc}

    out_input_test = sess.run(outputs_orig, input_test)

    for i in range(a.shape[0]):
        target_fn, mask_out = get_target_np(out_input_test, a[i], show_img=False)
        
        
        alpha_to_graph = int(np.round(a[i] / eps))
        
        direction = np.sign(alpha_to_graph)
        steps = np.abs(alpha_to_graph)
        single_step_alpha = num_steps + direction
        # within the graph range, we can compute it directly
        if 0 <= alpha_to_graph + num_steps <= num_steps * 2:
            zs_out = sess.run(z_new, {
                z:zs[s], alpha:
                alpha_to_graph + num_steps})
        else:
            # recursive step
            zs_next = zs[s]
            for n in range(steps):
                feed_dict = {z: zs_next, alpha: single_step_alpha}
                zs_next = sess.run(z_new, feed_dict=feed_dict)
            zs_out = zs_next
        # already taken n steps at this point, so directly use zs_next
        # without any further modifications: using num_steps index into w
        # alternatively, could also use outputs_orig
        best_inputs = {z: zs_out, y: ys[s],
                       truncation: trunc, alpha: num_steps}
        best_im_out = sess.run(transformed_output, best_inputs)
        
        # collect images
        ims.append(np.uint8(np.clip(((best_im_out + 1) / 2.0) * 256, 0, 255)))
        targets.append(np.uint8(np.clip(((target_fn + 1) / 2.0) * 256, 0, 255)))
        
    im_stack = np.concatenate(targets + ims).astype(np.uint8)
    imshow(imgrid(im_stack, cols = len(a)))

In [ ]:
# plot losses 
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(loss_vals)
plt.xlabel('num samples, lr{}'.format(lr))
plt.ylabel('Loss')
plt.show()